## Instagram Scraper

In [75]:
# import dependencies 
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np

In [109]:
# following this tutorial 
# https://medium.com/@srujana.rao2/scraping-instagram-with-python-using-selenium-and-beautiful-soup-8b72c186a058

username = 'pickuplimes'

# windows browser/webdriver
browser = webdriver.Chrome('C:\\Users\\Trisarah\\Desktop\chromedriver.exe')
browser.get('https://www.instagram.com/' + username + '/?hl=en')
Pagelength = browser.execute_script("window.scrollTo(0,document.body.scrollHeight);")

In [ ]:
# executable_path = {'executable_path': 'chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=False)

## mac users

In [ ]:
# executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
# browser = Browser('chrome', **executable_path, headless=False)

## location url

In [108]:
# location url 
# https://www.instagram.com/explore/locations/<short_name>/<museum_name>/
# https://www.instagram.com/explore/locations/26131/newseum/

In [78]:
# parse html source
links = []
source = browser.page_source
data = bs(source, 'html.parser')
body = data.find('body')
script = body.find('span')
for link in script.findAll('a'):
    if re.match("/p", link.get('href')):
        links.append('https://www.instagram.com' + link.get('href'))

In [82]:
num_comments = []
num_likes = []

result = pd.DataFrame()
for i in range(len(links)):
    try:
        page = urlopen(links[i]).read()
        data = bs(page, 'html.parser')
        body = data.find('body')
        script = body.find('script')
        raw = script.text.strip().replace('window._sharedData =', '').replace(';', '')
        json_data = json.loads(raw)
        posts = json_data['entry_data']['PostPage'][0]['graphql']
        comments = posts['shortcode_media']['edge_media_to_parent_comment']['count']
        num_comments.append(comments)
        likes = posts['shortcode_media']['edge_media_preview_like']['count']
        num_likes.append(likes)
       
    except:
        np.nan
# Just check for the duplicates
# result = result.drop_duplicates(subset = 'shortcode')
result.index = range(len(result.index))

In [104]:
# do not want to grab the first ig link since it is the username...
# when i do this it cuts off the last one too for some reasonnnnnn
links[1:-1]

['https://www.instagram.com/p/B1v5eD-oljJ/',
 'https://www.instagram.com/p/B1q8f1yoTAQ/',
 'https://www.instagram.com/p/B1g7TmkIOe7/',
 'https://www.instagram.com/p/B1ZbAwVIXFs/',
 'https://www.instagram.com/p/B1G40TxIWBp/',
 'https://www.instagram.com/p/B08exSQIdQB/',
 'https://www.instagram.com/p/B03vCmCojuf/',
 'https://www.instagram.com/p/B01cvNzIX43/',
 'https://www.instagram.com/p/B0dyKQKodYF/',
 'https://www.instagram.com/p/B0V80A-Ic13/',
 'https://www.instagram.com/p/B0T5tEhIj6H/',
 'https://www.instagram.com/p/Bz-3QHpIDOe/',
 'https://www.instagram.com/p/BzvcPzRofFC/',
 'https://www.instagram.com/p/BzsrYMwITHM/',
 'https://www.instagram.com/p/BzquhV9ITm2/',
 'https://www.instagram.com/p/BzoDlQJoovx/',
 'https://www.instagram.com/p/BzlHBDAIuZa/',
 'https://www.instagram.com/p/BzRCtfcIWG6/',
 'https://www.instagram.com/p/Bysh2YvoV8v/',
 'https://www.instagram.com/p/ByQpQOFIoy7/',
 'https://www.instagram.com/p/ByK786vIJeX/',
 'https://www.instagram.com/p/ByDMPMQIrDZ/',
 'https://

In [105]:
# create insta df
insta_df = pd.DataFrame(
    {'links': links[1:25],
     'number of comments': num_comments, 
     'number of likes': num_likes
    })
insta_df

,links,number of comments,number of likes
0,https://www.instagram.com/p/B1v5eD-oljJ/,135,10009
1,https://www.instagram.com/p/B1q8f1yoTAQ/,746,28316
2,https://www.instagram.com/p/B1g7TmkIOe7/,156,15812
3,https://www.instagram.com/p/B1ZbAwVIXFs/,255,17306
4,https://www.instagram.com/p/B1G40TxIWBp/,244,20037
5,https://www.instagram.com/p/B08exSQIdQB/,249,14061
6,https://www.instagram.com/p/B03vCmCojuf/,2445,15457
7,https://www.instagram.com/p/B01cvNzIX43/,725,34161
8,https://www.instagram.com/p/B0dyKQKodYF/,247,14992
9,https://www.instagram.com/p/B0V80A-Ic13/,216,15890
